#**El proyecto realiza la clasificacion de textos en PDF a travez del analisis del texto con distintos modelos**

In [ ]:
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 77.3 MB/s eta 0:00:00


In [ ]:
import re
import io
import pandas as pd
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams

def extract_text_from_pdf(file_path):
    resource_manager = PDFResourceManager()
    text_stream = io.StringIO()
    laparams = LAParams()
    converter = TextConverter(resource_manager, text_stream, laparams=laparams)
    interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(file_path, 'rb') as file:
        for page in PDFPage.get_pages(file, check_extractable=True):
            interpreter.process_page(page)

    text = text_stream.getvalue()
    converter.close()
    text_stream.close()

    # Limpiar el texto
    text = re.sub(r'\s+', ' ', text)  # Eliminar espacios en blanco adicionales
    text = re.sub(r'\n', ' ', text)  # Eliminar saltos de línea
    text = re.sub(r'\r', ' ', text)  # Eliminar retornos de carro

    return text.strip()

def create_dataset():
    # Autenticación con Google Drive
    auth.authenticate_user()

    # Construir el servicio de Google Drive
    drive_service = build('drive', 'v3')

    # Obtener los archivos de la carpeta 'Docs' en Mi unidad de Google Drive
    folder_name = 'Docs'
    folder_query = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder'"
    folders = drive_service.files().list(q=folder_query).execute().get('files', [])

    if not folders:
        print(f"No se encontró la carpeta '{folder_name}' en Mi unidad.")
        return None

    folder_id = folders[0]['id']

    # Crear el dataset
    dataset = pd.DataFrame(columns=['texto', 'categoria'])

    # Obtener los archivos PDF de la carpeta
    pdf_query = f"'{folder_id}' in parents and mimeType='application/pdf'"
    pdf_files = drive_service.files().list(q=pdf_query).execute().get('files', [])

    for pdf_file in pdf_files:
        file_name = pdf_file['name']
        category = re.findall('[A-Za-z]+', file_name)[0]  # Extraer la categoría del nombre del archivo

        # Descargar el archivo PDF
        file_path = f"/tmp/{file_name}"
        request = drive_service.files().get_media(fileId=pdf_file['id'])
        fh = io.FileIO(file_path, 'wb')
        downloader = MediaIoBaseDownload(fh, request)
        done = False

        while not done:
            status, done = downloader.next_chunk()

        # Extraer el texto del archivo PDF
        text = extract_text_from_pdf(file_path)

        # Agregar el texto y la categoría al dataset
        dataset = dataset.append({'texto': text, 'categoria': category}, ignore_index=True)

    return dataset

# Ejecutar la función create_dataset() para obtener el dataset
dataset = create_dataset()

if dataset is not None:
    print(dataset)


<ipython-input-9-debc42efdbc8>:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'texto': text, 'categoria': category}, ignore_index=True)
<ipython-input-9-debc42efdbc8>:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'texto': text, 'categoria': category}, ignore_index=True)
<ipython-input-9-debc42efdbc8>:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'texto': text, 'categoria': category}, ignore_index=True)
<ipython-input-9-debc42efdbc8>:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'texto': text, 'categoria': category}, ign

                                                texto categoria
0   (cid:2)(cid:3)(cid:4)(cid:3)(cid:5)(cid:6)(cid...        PI
1   Evaluación de daños en pavimento flexible usan...        PI
2   Simulación de entorno 3D y cálculo a punto en ...        PI
3   Ingeniería y Competitividad, Volumen 14, No. 2...        PI
4   15-22 Clasificación de los frutos de café segú...        PI
5   PROSPECTIVA Vol. 16 - No. 2 / Julio - Diciembr...        PI
6   Revista Ciencia en Desarrollo, Vol. 6 No. 1 IS...        PI
7   Printed online ahead of print Revista Ingenier...        PI
8   Revista Ingeniería Biomédica ISSN 1909-9762 / ...        PI
9   Revista EIA, ISSN 1794-1237 Número 6, p. 43-55...        PI
10  Revista Cubana de Ciencias Informáticas Vol. 9...        AA
11  Procesamiento del Lenguaje Natural, núm. 35 (2...        AA
12  http://revistas.unam.mx/index.php/entreciencia...        AA
13  ANÁLISIS ANÁLISIS SUPERVISADO DE SENTIMIENTOS ...        AA
14  Revista Cubana de Ciencias Informáti

<ipython-input-9-debc42efdbc8>:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'texto': text, 'categoria': category}, ignore_index=True)


In [ ]:
dataset.head(40)


,texto,categoria
0,(cid:2)(cid:3)(cid:4)(cid:3)(cid:5)(cid:6)(cid...,PI
1,Evaluación de daños en pavimento flexible usan...,PI
2,Simulación de entorno 3D y cálculo a punto en ...,PI
3,"Ingeniería y Competitividad, Volumen 14, No. 2...",PI
4,15-22 Clasificación de los frutos de café segú...,PI
5,PROSPECTIVA Vol. 16 - No. 2 / Julio - Diciembr...,PI
6,"Revista Ciencia en Desarrollo, Vol. 6 No. 1 IS...",PI
7,Printed online ahead of print Revista Ingenier...,PI
8,Revista Ingeniería Biomédica ISSN 1909-9762 / ...,PI
9,"Revista EIA, ISSN 1794-1237 Número 6, p. 43-55...",PI


In [ ]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.5 MB/s eta 0:00:00


In [ ]:
!pip install pdfminer.six

In [ ]:
import re
import os
import pandas as pd
from google.colab import drive
from PyPDF2 import PdfReader

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        num_pages = len(pdf_reader.pages)

        text = ''
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    # Limpiar el texto
    text = re.sub(r'\s+', ' ', text)  # Eliminar espacios en blanco adicionales
    text = re.sub(r'\n', ' ', text)  # Eliminar saltos de línea
    text = re.sub(r'\r', ' ', text)  # Eliminar retornos de carro

    return text.strip()

def create_dataset():
    # Montar Google Drive
    drive.mount('/content/drive')

    # Ruta de la carpeta 'Docs' en Google Drive
    folder_path = '/content/drive/MyDrive/Docs'

    # Verificar si la carpeta 'Docs' existe
    if not os.path.exists(folder_path):
        print("No se encontró la carpeta 'Docs' en la ruta especificada.")
        return None

    # Crear el dataset
    dataset = pd.DataFrame(columns=['texto', 'categoria'])

    # Obtener los archivos PDF de la carpeta
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        # Verificar si es un archivo PDF
        if os.path.isfile(file_path) and file_path.lower().endswith('.pdf'):
            file_name = os.path.splitext(file_name)[0]
            category = re.findall('[A-Za-z]+', file_name)[0]  # Extraer la categoría del nombre del archivo

            # Extraer el texto del archivo PDF
            text = extract_text_from_pdf(file_path)

            # Agregar el texto y la categoría al dataset
            dataset = dataset.append({'texto': text, 'categoria': category}, ignore_index=True)

    return dataset

# Ejecutar la función create_dataset() para obtener el dataset
dataset = create_dataset()

if dataset is not None:
    print(dataset)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-16-203a5137bb47>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'texto': text, 'categoria': category}, ignore_index=True)
<ipython-input-16-203a5137bb47>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'texto': text, 'categoria': category}, ignore_index=True)
<ipython-input-16-203a5137bb47>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'texto': text, 'categoria': category}, ignore_index=True)
<ipython-input-16-203a5137bb47>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'texto': text, 'categoria': category},

                                                texto categoria
0   REVISTA TECNOLÓGICA N° 11. ENERO - DICIEMBRE 2...        AA
1   Metodología de Aprendizaje Automático para la ...        AA
2   Clasificación de hongos por medio de aprendiza...        AA
3   Clasificación de la ocupación espectral para l...        AA
4   Revista C ubana de Ciencias Informáticas ISSN:...        AA
5   El profesional de la información , 2017, septi...        AA
6   DOI: 10.22201/enesl.20078064e.2018.19.67511 © ...        AA
7   NERUA: sistema de detecci´ on y clasiﬁcaci´ on...        AA
8   Revista Cubana de Ciencias Informáticas Vol. 9...        AA
9   Artículo recibido 15-III-2006. Aprobado 30-XI-...        PI
10  Revista Ingeniería Biomédica ISSN 1909-9762 / ...        PI
11  Roa E. et al. Viabilidad espermática mediante ...        PI
12  Revista Ciencia en Desarrollo, Vol. 6 No. 1 IS...        PI
13  PROSPECTIVA Vol. 16 - No. 2 / Julio - Diciembr...        PI
14  15 Prospect. Vol. 14, No. 1, Enero -

<ipython-input-16-203a5137bb47>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'texto': text, 'categoria': category}, ignore_index=True)


In [ ]:
import nltk
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Cargar el modelo de lenguaje en español de spaCy
nlp = spacy.load('es_core_news_sm')

# Entrenar y clasificar con Multinomial Naive Bayes
nb_model = nltk.NaiveBayesClassifier.train(training_set)
nb_predictions = nb_model.classify_many(test_set)
nb_accuracy = accuracy_score(test_labels, nb_predictions)
print("Multinomial Naive Bayes Accuracy:", nb_accuracy)

# Entrenar y clasificar con Bernoulli Naive Bayes
bnb_model = nltk.BernoulliNB.train(training_set)
bnb_predictions = bnb_model.classify_many(test_set)
bnb_accuracy = accuracy_score(test_labels, bnb_predictions)
print("Bernoulli Naive Bayes Accuracy:", bnb_accuracy)

# Entrenar y clasificar con Regresión Logística
lr_model = nltk.classify.scikitlearn.SklearnClassifier(LogisticRegression())
lr_model.train(training_set)
lr_predictions = lr_model.classify_many(test_set)
lr_accuracy = accuracy_score(test_labels, lr_predictions)
print("Logistic Regression Accuracy:", lr_accuracy)

# Entrenar y clasificar con Support Vector Machine (SVM)
svm_model = nltk.classify.scikitlearn.SklearnClassifier(SVC())
svm_model.train(training_set)
svm_predictions = svm_model.classify_many(test_set)
svm_accuracy = accuracy_score(test_labels, svm_predictions)
print("Support Vector Machine Accuracy:", svm_accuracy)

# Entrenar y clasificar con Random Forest
rf_model = nltk.classify.scikitlearn.SklearnClassifier(RandomForestClassifier())
rf_model.train(training_set)
rf_predictions = rf_model.classify_many(test_set)
rf_accuracy = accuracy_score(test_labels, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)
